# Goal
Label the mixed dataet as:
- $s = 0$: real
- $s = 1$: synthetic

Then train an RF to predict $s$ from the features $x$:
$$
\hat{p}(x) \;=\; \widehat{P}(s=1 \mid x),
$$
i.e., the predicted probability that an observation is synthetic.

We then evaluate separability using AUC computed from ($\hat{p}(x)$) on a held-out mixed test set:
- AUC ($\approx 0.5$): RF cannot distinguish real vs synthetic (desired)
- AUC ($> 0.5$): RF can distinguish the domains (synthetic differs from real)

#### Note: 
Because the dataset is small for HIV+ ($n = 68$) and HIV- ($n = 23$), we are going to **stratify sample**:
1. Split the data into stratas (groups)
2. Then sample insid each stratum separately


Naively we might sample holdouts as:
```python
rng.choice(all_indices, size=15)
```
The main issue is that HIV+ and HIV- are expexcted to have unsimilar values across all features.

And because the real and synthetic pools can each have different HIV−/HIV+ proportions when sampled, this runs the risk of RF detecting class-composition differences rather than feature-level differences between real and synthetic.

So we instead sample **stratified** holdouts:
```python
sample 3 from HIV-
sample 12 from HIV+
combine them
```
Stratification enforces equal HIV composition across domains, so any separability detected by the RF must arise from differences in feature distributions within HIV strata rather than from HIV prevalence imbalance.
$$
H_{0} : p(x \mid s = 0)  = P(x \mid s = 1)
$$.

HIV status is a structured covariate that influences feature distributions.
Stratification equalizes its prevalence across domains so that domain separability reflects within-stratum feature differences rather than mixture imbalance.

### Libraries

In [153]:
import numpy as np
import pandas as pd
from IPython.display import  display, Markdown, HTML
rng = np.random.default_rng(42)
np.set_printoptions(suppress=True, precision=3)

## Helper Functions

### Helper 1: Loading Data

In [115]:
# Real data
import pyreadr
original_data = pyreadr.read_r("../data/allSyntheticData.RData")
x_o = original_data["x"]
y_o = original_data["y"]

print(x_o.shape) # 91, 63
print((y_o == 1).sum()) # 68 HIV+, 23 HIV- 

X_real = np.asarray(x_o)
y_real = np.asarray(y_o).flatten().astype(int)

(91, 63)
[0. 1.]


In [106]:
# Synthetic data merging (HIV+ and HIV- were generated separately, so we need to merge them together)
syn0 = np.load("../data/output/cvae_synth_seed42_y0.npz")
syn1 = np.load("../data/output/cvae_synth_seed42_y1.npz")

x_syn0 = syn0["X"] # 91, 63
x_syn1 = syn1["X"] # 91, 63
X_syn = np.vstack([x_syn0, x_syn1])

# We need to add the labelss HIV- = 0, HIV+ = 1
y_syn = np.concatenate([np.zeros(x_syn0.shape[0]), np.ones(x_syn1.shape[0])]).astype(int)

# __________________________________________________________
# NB visual
col_names = original_data["x"].columns
# print(col_names)
cvae_df = pd.DataFrame(X_syn, columns=col_names)
df_left1 = cvae_df.iloc[:5, :5] 
df_right1 = pd.DataFrame({
    "Label": y_syn[:5]
})
display(
    Markdown("### First 5 rows of synthetic data (HIV-)"),
    HTML(
    "<div style='display:flex; gap:40px;'>"
    f"<div>{df_left1.to_html(index=False)}</div>"
    f"<div>{df_right1.to_html(index=False)}</div>"
    "</div>"
))
df_left2 = cvae_df.iloc[-5:, :5] 
df_right2 = pd.DataFrame({
    "Label": y_syn[-5:]
})
display(
    Markdown("### Last 5 rows of synthetic data (HIV+)"),
    HTML(
    "<div style='display:flex; gap:40px;'>"
    f"<div>{df_left2.to_html(index=False)}</div>"
    f"<div>{df_right2.to_html(index=False)}</div>"
    "</div>"
))


### First 5 rows of synthetic data (HIV-)

### Last 5 rows of synthetic data (HIV+)

### Helper 2: Stratified Sampler

In [104]:
def strat_samp(neg_i, posi_i, n_neg, n_posi):
    chosen_neg = rng.choice(neg_i, size = n_neg, replace = False)
    chosen_posi = rng.choice(posi_i, size = n_posi, replace = False)
    return np.concatenate([chosen_neg, chosen_posi])

# Example
neg_i = np.where(y_real == 0)[0]
posi_i = np.where(y_real == 1)[0]
sampled_indices = strat_samp(neg_i, posi_i, n_neg=5, n_posi=5)
print("Sampled indices:", sampled_indices)

Sampled indices: [ 8  7 10 14  2 70 65 77 33 50]


## Random Forest
One stochastic Random Forest instance contains:
- Test holdout = $3 \text{ HIV-}$, $12 \text{ HIV+}$ one from real, and another from synthetic
- Training = $20 \text{ HIV-}$, $20 \text{ HIV+}$ one from real, and another from synthetic
- real : $0$, syn : $1$
- RF 200 trees

In [119]:
from sklearn.ensemble import RandomForestClassifier

def train_rf(X_train, s_train):
    rf = RandomForestClassifier(n_estimators = 200, random_state = 42, n_jobs=-1)
    rf.fit(X_train, s_train)
    return rf

## AUC
With our labels $s$, RF will producee a score (probability):
$$
\hat{p}(x) = \hat{P}(s= 1\mid x)
$$

Then the empirical AUC is:
$$
AUC = \frac{1}{n_{s=1}\cdot n_{s=0} } \sum_{i:s_{i}=1} \sum_{j:s_{j}=0} 1(\hat{p_{i}} > \hat{p_{j}})
$$
The probability that a randomly chosen synthetic sample receives a higher predicted score than a randomly chosen real sample.

For every synthetic–real pair, we:

1. Compare their predicted probabilities

2. Count 1 if synthetic has higher score

3. Count 0 otherwise

4. Average over all pairs

*Note: If `synthetic score == real score` count 0.5*

In [ ]:
from sklearn.metrics import roc_auc_score

def get_auc(model, X_test, s_test):
    probs = model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
    auc = roc_auc_score(s_test, probs)
    return auc

# Running one stochastic experiment

In [ ]:
def one_stochastic_experiment(
    X_real, y_real_hiv,
    X_syn,  y_syn_hiv,
    holdout_neg=3, holdout_pos=12,
    train_neg=20,  train_pos=20,
):

    #strat sample
    real_neg = np.where(y_real_hiv == 0)[0]
    real_pos = np.where(y_real_hiv == 1)[0]
    syn_neg  = np.where(y_syn_hiv  == 0)[0]
    syn_pos  = np.where(y_syn_hiv  == 1)[0]

    #test/holdout: 3:12 
    test_real_idx = strat_samp(real_neg, real_pos, holdout_neg, holdout_pos)
    test_syn_idx  = strat_samp(syn_neg,  syn_pos,  holdout_neg, holdout_pos)

    # remaining indices for training 
    rem_real = np.setdiff1d(np.arange(X_real.shape[0]), test_real_idx)
    rem_syn  = np.setdiff1d(np.arange(X_syn.shape[0]),  test_syn_idx)

    rem_real_neg = rem_real[y_real_hiv[rem_real] == 0]
    rem_real_pos = rem_real[y_real_hiv[rem_real] == 1]
    rem_syn_neg  = rem_syn[y_syn_hiv[rem_syn] == 0]
    rem_syn_pos  = rem_syn[y_syn_hiv[rem_syn] == 1]


    train_real_idx = strat_samp(rem_real_neg, rem_real_pos, train_neg, train_pos)
    train_syn_idx  = strat_samp(rem_syn_neg,  rem_syn_pos,  train_neg, train_pos)


    # Real vs. Syn -classification dataset prep
    X_train = np.vstack([X_real[train_real_idx], X_syn[train_syn_idx]])
    s_train = np.concatenate([np.zeros(len(train_real_idx), dtype=int),
                              np.ones(len(train_syn_idx), dtype=int)])

    X_test  = np.vstack([X_real[test_real_idx],  X_syn[test_syn_idx]])
    s_test  = np.concatenate([np.zeros(len(test_real_idx), dtype=int),
                              np.ones(len(test_syn_idx), dtype=int)])


    rf = train_rf(X_train, s_train)

    p_syn = rf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(s_test, p_syn)
    display(Markdown(f"### AUC (one iteration): {auc:.4f}"))
    print("Train shape:", X_train.shape)
    print("Test shape:", X_test.shape)

    print("Train real count:", (s_train == 0).sum())
    print("Train syn count:", (s_train == 1).sum())

    print("Test real count:", (s_test == 0).sum())
    print("Test syn count:", (s_test == 1).sum())

    print("Mean real (test):", X_test[s_test==0].mean())
    print("Mean syn  (test):", X_test[s_test==1].mean())
    print("Std real:", X_test[s_test==0].std())
    print("Std syn :", X_test[s_test==1].std())
    return auc

# Okay, so that's not good.
*AUC (one iteration): 0.9333333333333333*

In [ ]:

real_neg = np.where(y_real == 0)[0]
real_pos = np.where(y_real == 1)[0]
syn_neg  = np.where(y_syn  == 0)[0]
syn_pos  = np.where(y_syn  == 1)[0]

#test/holdout: 3:12 
test_real_idx = strat_samp(real_neg, real_pos, 3, 12)
test_syn_idx  = strat_samp(syn_neg,  syn_pos,  3, 12)

# remaining indices for training 
rem_real = np.setdiff1d(np.arange(X_real.shape[0]), test_real_idx)
rem_syn  = np.setdiff1d(np.arange(X_syn.shape[0]),  test_syn_idx)

rem_real_neg = rem_real[y_real[rem_real] == 0]
rem_real_pos = rem_real[y_real[rem_real] == 1]
rem_syn_neg  = rem_syn[y_syn[rem_syn] == 0]
rem_syn_pos  = rem_syn[y_syn[rem_syn] == 1]


train_real_idx = strat_samp(rem_real_neg, rem_real_pos, 20, 20)
train_syn_idx  = strat_samp(rem_syn_neg,  rem_syn_pos,  20,  20)


# Real vs. Syn -classification dataset prep
X_train = np.vstack([X_real[train_real_idx], X_syn[train_syn_idx]])
s_train = np.concatenate([np.zeros(len(train_real_idx), dtype=int),
                            np.ones(len(train_syn_idx), dtype=int)])

X_test  = np.vstack([X_real[test_real_idx],  X_syn[test_syn_idx]])
s_test  = np.concatenate([np.zeros(len(test_real_idx), dtype=int),
                            np.ones(len(test_syn_idx), dtype=int)])


rf = train_rf(X_train, s_train)

p_syn = rf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(s_test, p_syn)




### AUC (one iteration): 0.9822

Train shape: (80, 63)
Test shape: (30, 63)
Train real count: 40
Train syn count: 40
Test real count: 15
Test syn count: 15
Mean real (test): 442.36580776506173
Mean syn  (test): 367.5676839438676
Std real: 891.8657762666176
Std syn : 641.4845764003218


- The location (means) are so far away
- Scale is basically the same (stdev)

# Feature-wise means

In [171]:
mean_diff = X_syn.mean(axis=0) - X_real.mean(axis=0)

df_diff = pd.DataFrame({
    "feature": col_names,
    "mean_diff": mean_diff
})

df_diff_sorted = df_diff.reindex(
    df_diff["mean_diff"].abs().sort_values(ascending=False).index
)
df_diff_sorted["mean_diff"] = df_diff["mean_diff"].round(1)

display(df_diff_sorted.head(25)) 
display(Markdown("\\+ is favour synthetic, - is favour real"))


,feature,mean_diff
26,V10_blood_IgGRBD,-481.4
25,V9_blood_IgGRBD,-422.7
58,V9Neut,-233.2
16,V10_blood_IgGspike,-227.4
21,V6_blood_IgGRBD,216.0
27,V11_blood_IgGRBD,176.6
62,V9_ACE2,-138.4
57,V8Neut,120.8
24,V8b_blood_IgGRBD,76.6
23,V8a_blood_IgGRBD,-73.3


\+ is favour synthetic, - is favour real